# Audience
Пример отчета по расчету аудитории интернет-проектов (WEB-Index).

## Описание задачи

Посчитаем аудиторию по интернет-проектам:
- данные по одному проекту Avito.ru
- данные по двум проектам Avito.ru, Ivi.ru, т.е. количество людей, которые посетили хотя бы один проект (Avito.ru)OR(Ivi.ru)
- данные по Total Internet - Total

Общие параметры:
- Период: Май 2020
- География: Россия 0+
- Население: 12+
- Тип использования: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline)

Статистики:
- Unweighted Reach (UnwReach)
- Reach
- Reach% (ReachPer)
- Average Daily Reach (ADR)
- Average Daily Reach% (ADRPer)
- Average Weekly Reach (AWR)
- Average Weekly Reach% (AWRPer)

Для расчета данного отчета потребуется сформировать несколько заданий для Responsum.
В отчете присутствуют кастомные переменные (Avito.ru, Ivi.ru), а также Total Internet - все это отдельные задания.


## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

## Формируем задания

Необходимо сформировать три задания для Responsum:
- расчет аудитории проекта Avito.ru
- расчет совокупной аудитории проектов Avito.ru, Ivi.ru
- расчет аудитории Total Internet

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile, т.к. в расчетах участвуют все типы устройств (Web Desktop, Web Mobile, App Online, App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Получим ID проектов
Для построения отчета необходимо получить идентификаторы сайтов __Avito.ru__ и __Ivi.ru__.
Для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb#catalogs_site_avito):

- Avito.ru  site_id = 15828
- Ivi.ru  site_id = 15869


## Задания

Перейдем к формированию заданий

# Дополнительный фильтр по демографии отсутствует
media_filter = None

### Задание №1. Расчет аудитории проекта Avito.ru

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'avito.ru'

# задаем медиа фильтр, в нашем случае это ID проекта Avito.ru
media_filter = "site = 15828"

# дополнительный фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False,
    }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_avito = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_avito

### Задание №2. Расчет совокупной аудитории проектов Avito.ru и Ivi.ru

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'avito.ru+ivi.ru'

# задаем медиа фильтр, в нашем случае это ID проектов Avito.ru и Ivi.ru
media_filter = rtask.get_sql_from_list("site", [15828, 15869])

# дополнительный фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False,
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_avito_ivi = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_avito_ivi

### Задание №3. Расчет аудитории Total Internet

Расчет данных по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.
Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame

In [ ]:
# объединим наши DataFrame'ы
df_result = pd.concat([df_avito, df_avito_ivi, df_total])
df_result

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('audience-avito-ivi'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()